# Implementing Electronic Stopping in Molecular Dynamics codes

The easiest way to include the effect of electronic stopping in a molecular dynamics (MD) code is by damping the velocity of an atom by a viscous force: $$F_{es} = \beta v_i,$$ where $\beta$ is the drag co-efficient having units of mass/time, and, $v_i$ is the velocity of the atom $i$ [1]. The value of $\beta$ can be obtained by several ways: <br>
1. Firsov's model of mean electronic excitations in atomic collisions [2]
2. Lindhard and Scharff's formula for energy dissipation of energetic ions [3]
3. Finnis, et al's. formula taking into account the energy transfer from ions to a background electron gas at a specified temperature [4]
4. Caro et al's. formula including local density effects which can be implemented for interatomic potentials that use the local electron density [5], and
5. Nordlund, et al., who use SRIM [6] data for the electronic stopping [7].

## Incorporation of viscous drag in MD
Before getting into the specifc models, a discussion of how a viscous drag is implemented within the MD algorithm is essential. The stopping force due to electronic drag by a target on an incoming atom having a kinetic energy $E$ is given by 
\begin{equation}
\label{ForceES}
\tag{1}
F_{es} = \frac{dE}{dx} = n S_{es}(E)
\end{equation}
where, $S_{es}$ is the electronic stopping cross section per target atom and n is the number density of the target atoms.
\begin{eqnarray}
\label{dEbydxRewrite}
\tag{2}
\frac{dE}{dx} &=& \frac {dE}{dt}\frac {dt}{dx}
              &=& \frac {1}{v} \frac{dE}{dt}
\end{eqnarray}
where, $v$ is the velocity of the energetic atom.
From the above two equations we get:
\begin{equation}
\label{dEbydt1}
\tag{3}
\frac{dE}{dt} = n v S_{es}
\end{equation}
Substituting $E = 0.5\ m\ v^2$ in the above equation we get
\begin{equation}
\label{dvbydt}
\tag{4}
m \frac{dv}{dt} = n S_{es}
\end{equation}


## Models for the electronic stopping power

### Firsov's model

### Lindhard's model
In Lindhard's model, we have
\begin{equation}
\label{LSStope}
\tag{5}
S_{es}(E) = 8 \pi e^2 a_o Z_1^{1/6} \frac {Z_1 Z_2}{(Z_1^{2/3}+Z_2^{2/3})^{3/2}} \frac{v}{v_o} = \lambda v
\end{equation}
where,
$$\lambda = 8 \pi e^2 a_o Z_1^{1/6} \frac {Z_1 Z_2}{(Z_1^{2/3}+Z_2^{2/3})^{3/2} v_o}$$
$Z_1$ and $Z_2$ are the atomic numbers of the energetic atom and of the target respectively, $a_o$ is the Bohr radius, $e$ is the electronic charge and $v_o$ is the fermi velocity for the target atoms.
The Fermi velocity is the velocity corresponding to the Fermi Energy given by:
\begin{eqnarray}
\label{FermiEngy}
\tag{6}
E_o & = & \frac{\hbar^2}{2m_e} (3 \pi^2 Ne_{val}*n)^{2/3}
\end{eqnarray}
where $m_e$ is the electron mass, $Ne_{val}$ is the number of electrons in the outermost shell of the atom and n is the number density of the target atoms. n can easily be obtained from the lattice constant $c$ and the number of atoms in an unit cell $n_{at}$ using $$n=n_{at}/c^3$$

The Fermi velocity is then:
\begin{eqnarray}
\label{FermiVel}
\tag{7}
v_o & = & \sqrt{\left ( \frac {2.0*E_o}{m_e} \right )}
    & = & 5.9310e5 \sqrt{E_o} \ \ \  (in\ units\ of\ m/sec)
\end{eqnarray}

We therefore substitute Eqn.\ref{LSStope} in Eqn.\ref{dvbydt} to get
\begin{eqnarray}
\label{dvbydtestop}
\tag{8}
m \frac{dv}{dt} & = & n \lambda v \\ 
i.e\ F_{es}     & = & \beta v
\end{eqnarray}


### Finnis's model
### Caro's model

### Deriving the $\Delta E$ lost due to electronic stopping from MD (for checking with the electronic stopping losses from the NRT model - below:
$$
E = \frac{1}{2} m v^2
$$

\begin{equation}
\label{dEbydt2}
\tag{9}
\frac{dE}{dt} = mv\frac{dv}{dt}
\end{equation}
From Eqns.\ref{dEbydt1} and \ref{dEbydt2} we get
\begin{eqnarray}
\label{dEFromBeta}
\tag{10}
\frac{dE}{dt} = \beta v^2 \\
i.e. dE = \beta v^2 dt
\end{eqnarray}

### Finding $\beta$ for W

In [1]:
from __future__ import print_function, division # for backward compatibility with python2

import numpy as np
import scipy as sp
Z1 = 74 # a.m.u atomic number of energetic projectile
Z2 = 74 # a.m.u atomic number of target atoms
Wmass = 183.84 # grams/mole
h = 6.6261e-34 # (J-s)
pi = 3.14159265359
hcross = 1.0546e-34 # J-s
MassElectron = 9.1090e-31 #kgs
eVtoJoules = 1.6022e-19
WLatticeC = 3.16 # (Angstroms)
WnAtinUnitCell = 2 # Number of atoms in a unit cell of W
nW = WnAtinUnitCell/WLatticeC**3 * 1e30 # Number of atoms per m^3
NeW = 2 # Number of electrons in the valance shell of W
E_o = hcross**2/(2*MassElectron)*(3*pi**2*NeW*nW)**(2.0/3.0)/1.60221e-19
print ("Fermi Energy of W = %e (eV)" % (E_o))
v_o = 5.9310e5*np.sqrt(E_o)
print("Fermi Velocity of W = %e (Angstroms/ps)" % (v_o*1e-2))
a_o = 5.2917721067e-1 #(Angstroms)
# echarge = 0.37947 # (sqrt(eV-Angstrom))
echarge = 1.2 # (sqrt(eV-nm)) <-- modified on 2018-09-17
lamda = 8*pi*echarge**2*a_o*10*Z1**(1.0/6.0)*(Z1*Z2)/((Z1**(2.0/3.0)+Z2**(2.0/3.0))**(3.0/2.0)*v_o*1e-2)
print ("lamda = %e (eV-Angstrom-ps)" % lamda)
beta = nW*lamda/1e30
print ("beta = %e (ev ps mole / Angstrom^2 grams)" % beta)

Fermi Energy of W = 9.202417e+00 (eV)
Fermi Velocity of W = 1.799198e+04 (Angstroms/ps)
lamda = 5.706239e-01 (eV-Angstrom-ps)
beta = 3.616752e-02 (ev ps mole / Angstrom^2 grams)


### Varification of electronic stopping addition in MD
From the NRT model [8] find out the total energy loss due to electronic stopping $E_{PKA} - \hat{E}$.

$\hat{E}$ is given by:
$$ \hat{E} = \frac{E_{PKA}}{[1 + k g(\epsilon)]} $$, where
$$ g(\epsilon) = 3.4008\ \epsilon^{1/6}\ +\ 0.40244\ \epsilon^{3/4}\ +\ \epsilon $$,
$$ k = 0.1337\ Z_1^{1/6}\ \left (\frac{Z_1}{A_1}\right )^{1/2} $$
$$ \epsilon = \frac{A_2\ E_{PKA}}{A_1+A_2}\ \frac{a}{Z_1\ Z_2\ e^2} $$
where $A_1$ and $A_2$ are the atomic weights of the energetic atom in units of (grams/mole) and of the target atoms respectively. $e$ is the charge of an electron in units of $\sqrt{eV\ nm}$ and a is given by:<br> <br>
$$ a = \left( \frac{9\ \pi^2}{128} \right )^{1/3} \ a_o\ [Z_1^{2/3}\ +\ Z_2^{2/3}]^{-1/2} $$

In the MD code, obtain the total energy loss due to viscous damping term by summing up all the $\Delta E$ at each time step for all particles having a threshold energy, say > 5 eV, above which electronic stopping is assumed to be non-negligible. Compare the total loss due to viscous damping with $ E_{PKA} - \hat{E} $ for verifying the electronic stopping.



In [2]:
EPKA = 5000.0
Z1 = 74
A1 = 183.84
Z2 = 74
A2 = 183.84
a = 0.8854 * (a_o/10) / np.sqrt(Z1**(2/3)+ Z2**(2/3))
k = 0.1337 * Z1**(1/6) * np.sqrt(Z1/A1)
eps = A2 * EPKA / (A1+A2) * a / (Z1 * Z2 * echarge*echarge)
geps = 3.4008 * eps**(1/6) + 0.40244 * eps**(3/4) + eps
Ehat = EPKA/(1.0 + k * geps)
DeltaE = EPKA - Ehat
print("Delta E = %e eV" % DeltaE)

Delta E = 8.982784e+02 eV


<br>__References__ <br>
1. D. R. Mason et al, NIMPR-B, 269 (2011) 1640.
2. O.B. Firsov, Soviet Physics JETP, 36 (9) (1959) 1076.
3. J. Lindhard and M. Scharff, Phys. Rev. 124 (1) (1961) 128.
4. M.W. Finnis, P. Agnew and A.J.E. Foreman, Phys. Rev. B, 44 (2) (1991) 567.
5. A. Caro and M. Victoria, Phys. Rev. A, 40 (5) (1989) 2287.
6. J.F. Ziegker, J.P. Biersack and U. Littmark, "The stopping and Range of Ions in Solids", Pergammon Press, (1985).
7. K. Nordlund, et al., Phys. Rev. B, 57 (13) (1998) 7556.
8. M.J. Norgett, M.T. Robinson and I.M. Torrence, Nucl. Engg. and Design, 33 (1975) 50.
